In [1]:
import cntk 
import matplotlib.pyplot as plt
import numpy as np 
import time

In [2]:
from cntk.learners import sgd , learning_rate_schedule , UnitType 
from cntk.logging import ProgressPrinter
from cntk.layers import Dense , Sequential

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
print np.shape(mnist.train.images)
print np.shape(mnist.train.labels)
print np.shape(mnist.test.images)
print np.shape(mnist.test.labels)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [5]:
def generate_random_data(sample_size , feature_dim , num_classes):
    y = np.random.randint( size=(sample_size , 1 )  , low=0 , high = num_classes )
    x = (np.random.randn(sample_size , feature_dim) + 3 * (y+1))
    x = x.astype(np.float32)
    
    class_ind = [y == class_number for class_number in range(num_classes)]
    y= np.asarray(np.hstack(class_ind) , dtype=np.float32)
    return x,y

In [6]:
# Define a utility function to compute the moving average sum.
# A more efficient implementation is possible with np.cumsum() function
def moving_average(a, w=5):
    if len(a) < w:
        return a[:]    # Need to send a copy of the array
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]


# Defines a utility that prints the training progress
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose:
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))

    return mb, training_loss, eval_error

In [7]:
sample_size=3
num_classes=10
feature_dim=3
y = np.random.randint( size=(sample_size , 1 )  , low=0 , high = num_classes )
x = (np.random.randn(sample_size , feature_dim) + 3 * (y+1))
print np.random.randn(sample_size , feature_dim)
print y 
print x

[[-0.13840562 -0.39486715 -1.82903444]
 [-0.23992308 -0.34654054  1.1703136 ]
 [ 1.56885547  0.32146374 -0.53372104]]
[[7]
 [7]
 [1]]
[[ 22.74185795  24.08037805  23.62570173]
 [ 22.82202231  22.59268653  24.01240461]
 [  5.0079721    6.0367091    5.2138694 ]]


In [8]:
featrues=mnist.train.next_batch(60)
print np.shape(featrues[1])
print np.shape(featrues[0])

(60, 10)
(60, 784)


In [9]:
"""
fully connected classifier network model function 
ce = cross_entropy which defined out model's loss function 
pe = classification error 
"""

def ffnet():
    inputs = 784
    outputs = 10  
    layers =2 
    hidden_dimension=1024
    features = cntk.input_variable((inputs) , np.float32)
    print features
    label = cntk.input_variable((outputs) , np.float32)
    print label
    
    #[Dense(hidden_dimension ,activation=cntk.sigmoid)
    #Composite(Dense): Placeholder('x', [???], [???]) -> Output('Block16_Output_0', [???], [???])
    #Dense(outputs)
    
    print Dense(hidden_dimension ,activation=cntk.sigmoid)
    print Dense(outputs)
    my_model = Sequential(layers=[Dense(hidden_dimension ,activation=cntk.sigmoid),Dense(outputs)])
    print my_model 
    z = my_model(features)
    ce = cntk.cross_entropy_with_softmax(z , label)
    pe = cntk.classification_error(z, label)    
    
    lr_per_minibatch = learning_rate_schedule(0.125 , UnitType.minibatch)

    progress_printer = ProgressPrinter(0)
    trainer = cntk.Trainer(model=z , criterion=(ce , pe) , 
                           parameter_learners=[sgd(z.parameters ,lr=lr_per_minibatch)] ,
                       progress_writers=[progress_printer])
    # z 

    minibatch_size=60
    num_minibatches_to_train=300000
    aggregate_loss = 0.0
    s_time=time.time()
    
    for i in range(num_minibatches_to_train):

        #if you want to use 'generate_random_data' for data , uncomment below line
        #train_features , labels = generate_random_data(minibatch_size , inputs , outputs)

        train_images , train_labels=mnist.train.next_batch(60)
        trainer.train_minibatch({features : train_images  , label : train_labels })
        sample_count = trainer.previous_minibatch_sample_count
        aggregate_loss += trainer.previous_minibatch_loss_average * sample_count
        print_training_progress(trainer, i, 500, verbose=1)   
        if i ==0:
            print 'train features :', np.shape(train_images)
            print 'labels :' , np.shape(train_labels)
            print 'sample_count',sample_count
            print 'aggregate_loss',aggregate_loss
        
    print 'time : {}'.format(time.time()-s_time)
    last_avg_error = aggregate_loss / trainer.total_number_of_samples_seen
    #test_features , test_labels  = generate_random_data(minibatch_size , inputs , outputs)
    test_features=mnist.test.images
    test_labels = mnist.test.labels
    avg_error = trainer.test_minibatch({features : test_features , label : test_labels })
    print(' error rate on an unseen minibatch : {}'.format(avg_error))
    return last_avg_error , avg_error
    

np.random.seed(98052)
ffnet()

Input('Input3', [#], [784])
Input('Input4', [#], [10])
Dense(x: Sequence[tensor]) -> Sequence[tensor]
Dense(x: Sequence[tensor]) -> Sequence[tensor]

/usr/local/lib/python2.7/dist-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<type 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))



Composite(keep: Sequence[tensor]) -> Sequence[tensor]
 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.125
     2.75       2.75      0.933      0.933            60
Minibatch: 0, Loss: 2.7460, Error: 93.33%
train features : (60, 784)
labels : (60, 10)
sample_count 60
aggregate_loss 164.760940552
     4.17       4.88      0.911        0.9           180
     5.37       6.28      0.905        0.9           420
     5.48       5.56       0.89      0.877           900
     4.67       3.92      0.836      0.785          1860
     3.67       2.69      0.757       0.68          3780
      2.5       1.35      0.577      0.401          7620
     1.62      0.754      0.403      0.229         15300
Minibatch: 500, Loss: 0.3216, Error: 10.00%
     1.06      0.497      0.274      0.146         30660
Minibatch: 1000, Loss: 0.3402, Error: 11.67%
    0.73

KeyboardInterrupt: 

RuntimeError: SWIG director method error.